### More routing examples

$$
\begin{alignat*}{3}
\text{min  }  & \sum_{(i, j)} x_{i,j}D_{i,j} && \\
\text{subject to  }
& \sum_{j \in \Omega_{origin}^{out}} x_{origin, j} = 1\\
& \sum_{i \in \Omega_{dest}^{in}} x_{i, dest} = 1\\
& \sum_{j \in \Omega_{i}^{out}} x_{i, j} = \sum_{j \in \Omega_{i}^{in}} x_{j, i} 
&& \qquad \forall i \not {{origin, dest}}\\
\end{alignat*}
$$


In [1]:
import pyomo.environ as pe
import pyomo.opt as po
import pandas as pd

In [2]:
nodes_df = pd.read_excel("data.xlsx", sheet_name="nodes")
paths_df = pd.read_excel("data.xlsx", sheet_name="paths")
paths_df.index = range(1, paths_df.shape[0]+1)

In [3]:
m = pe.ConcreteModel()
m.I = pe.RangeSet(1, paths_df.shape[0])
m.x = pe.Var(m.I, domain=pe.Binary)

In [4]:
origin_node = nodes_df.node[nodes_df.desc == "origin"].item()
end_node = nodes_df.node[nodes_df.desc == "end"].item()

In [5]:
# Constraints

m.cons = pe.ConstraintList()
for node in nodes_df.node:
    in_index = paths_df[paths_df['f'] == node].index
    out_index = paths_df[paths_df['t'] == node].index

    if node  == origin_node:
        m.cons.add(sum(m.x[i] for i in in_index) == 1)
    elif node  == end_node:
        m.cons.add(sum(m.x[i] for i in out_index) == 1)
    else:
        m.cons.add(sum(m.x[i] for i in in_index) == sum(m.x[j] for j in out_index))
        
        
# Objective
m.obj = pe.Objective(
    expr=(sum(m.x[i]*paths_df.loc[i, "dis"] for i in m.I)),
    sense=pe.minimize
)

In [6]:
solver = po.SolverFactory('glpk')
results = solver.solve(m)

paths_df['activated'] = 0
for i in m.I:
    paths_df.activated[i] = pe.value(m.x[i])
print(pe.value(m.obj))


20.0


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16156\353106947.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paths_df.activated[i] = pe.value(m.x[i])


In [7]:
paths_df

,f,t,dis,activated
1,A,P1,2,0
2,A,P2,7,1
3,P1,P2,10,0
4,P2,P1,10,0
5,P1,B,30,0
6,P2,P3,8,1
7,P3,B,5,1
